## Chapter 12 exercises.

* testing out doing this as QMD instead of IPYNB. Not sure how well it will work though!


In [ ]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import bambi as bmb
import seaborn as sns

In [ ]:
eagles = pd.read_csv('bald_eagles.csv')
eagles.head()

In [ ]:
eagles.describe()

### Exercise 12.5

a) Look at count independant of the rest

In [ ]:
sns.histplot(eagles['count'])

b) Count vs year


In [ ]:
sns.scatterplot(data = eagles, x =  'year', y= 'count')

It does appear that the variance gets bigger as years go on, so a poissoin distribution could be tried.

c) Observation periods vary from year to year, from 134 to 248 hours. 

In [ ]:
sns.scatterplot(data = eagles, x =  'year', y= 'hours')

It looks like the spread in hours is rather uniform, but also that there is a jump around 2004, so that it is not independant of the year.

In [ ]:
sns.scatterplot(data = eagles, x =  'hours', y= 'count')

This tells us that hours have an (expected!) effect on the counts, so we will need to include (control for) that in our models.

### Exercise 12.7


In [ ]:
model1 = bmb.Model('count ~ year + hours', eagles, family='poisson')
model1.build()
model1.plot_priors()

:::{.callout-tip}
## Quarto issue

Needs cores=1 to avoid broken pipe error. Works fine in a notebook. Wierd!
:::

In [ ]:
results1 = model1.fit(draws=10000, chains=4,cores=1)
az.plot_trace(results1)
az.summary(results1)

Diagnostics look good, so moving on to look at the posterior check>

In [ ]:
model1.predict(results1, kind='pps', inplace= True)
az.plot_ppc(results1, num_pp_samples = 30)

This looks quite good to me.


### Exercise 12.8

In [ ]:
model2 = bmb.Model('count ~ year + hours', eagles, family='negativebinomial')
model2.build()
model2 

In [ ]:
model2.plot_priors()

I am not too sure what bambi is doing with `alpha` prior here, so once again i will use my own exponential prior


In [ ]:
alpha_prior = bmb.Prior('Exponential', lam= 1/50.)
model2 = bmb.Model('count ~ year + hours', eagles, family='negativebinomial',priors = {'alpha': alpha_prior})
model2.build()
model2 

In [ ]:
model2.plot_priors()

In [ ]:
results2 = model2.fit(draws=10000, chains=4,cores=1)
az.plot_trace(results2)
az.summary(results2)

Diagnostics look good, so moving on to look at the posterior check>

In [ ]:
model2.predict(results2, kind='pps', inplace= True)
az.plot_ppc(results2, num_pp_samples = 30)

Not sure it was worth all that extra trouble. Perhaps I am missing something,  because the count has a mean that is close to the standard deviation (across all years and hours). 

Based on this year and hours are both significant, but year is clearly the stronger predictor. 

Lets grab the raw samples so i can compute directly the quantile for the year coefficient.


In [ ]:
year_coeff_samples = results2.posterior.year.values.flatten()
np.quantile(year_coeff_samples,[.025,.975])

This tells us that , controlling for number of observation hours, we do have ample evidence that eagle sitings have increased over the years, on average by 8% a year:

In [ ]:
np.exp(np.mean(year_coeff_samples))-1.0

#### Ok i am going to redo this using `pymc` direct

In [ ]:
with pm.Model() as negBin:

    # define priors, weakly informative Normal
    b0 = pm.Normal("Intercept", mu=0, sigma=500)
    b1 = pm.Normal("year", mu=0, sigma=.3)
    b2 = pm.Normal("hours", mu=0, sigma=.1)
   # alpha = pm.Exponential("alpha",lam = 1/50.  )

    # define linear model and exp link function
    theta = (
        b0
        + b1 * eagles["year"].values
        + b2 * eagles["hours"].values
    )


    ## Define Poisson likelihood
    y = pm.Poisson("y", mu=pm.math.exp(theta), observed=eagles["count"].values)

In [ ]:
with negBin:
    samples = pm.sample(cores=2)

In [ ]:
az.plot_trace(samples);

### Exercise 12.9